In [1]:
# for data manipulation
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from gensim.models import Word2Vec
import gensim.downloader as api
from transformers import BertTokenizer, BertModel
import torch
import warnings
import scipy.sparse

# for data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# DL Libraries
import tensorflow as tf
from tensorflow import keras
from keras.layers import Conv2D , MaxPooling2D ,Dropout , Flatten , Dense ,BatchNormalization ,Concatenate ,Input 
from keras.models import Sequential ,Model
import json

# other libraries
import cv2
import os
import glob
from PIL import Image
import string
import re
from nltk.corpus import stopwords
from collections import Counter
from nltk.tokenize import WordPunctTokenizer
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.decomposition import PCA, TruncatedSVD  
from sklearn.naive_bayes import MultinomialNB
from sklearn.cluster import KMeans, AgglomerativeClustering, SpectralClustering
from sklearn.ensemble import RandomForestClassifier # Added RandomForestClassifier
from sklearn.metrics.cluster import adjusted_mutual_info_score, adjusted_rand_score # Changed metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score # Import classification metrics

# Model
import tensorflow as tf
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers, models, applications
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import MobileNetV2, ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, LabelEncoder
from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input
from sklearn.metrics import adjusted_mutual_info_score, adjusted_rand_score
from sklearn.svm import SVC
from scipy.stats import iqr # For Silverman's rule
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MultiLabelBinarizer

# RLAC MODEL
from sklearn.random_projection import GaussianRandomProjection
from scipy.stats import gaussian_kde
from scipy.signal import find_peaks  
from sklearn.neighbors import KernelDensity  
from sklearn.model_selection import GridSearchCV, LeaveOneOut
from scipy import stats
import diptest
from scipy.special import eval_hermitenorm  # For normalized Hermite polynomials H_n(x)
from scipy.stats import skew
from scipy.stats import norm
from sklearn.metrics import (adjusted_mutual_info_score, adjusted_rand_score, 
                             homogeneity_score, completeness_score, v_measure_score,
                             fowlkes_mallows_score, silhouette_score, calinski_harabasz_score,
                             davies_bouldin_score)

In [1]:
#import kagglehub

# Download latest version
#path = kagglehub.dataset_download("caesarlupum/vertebralcolumndataset")

#print("Path to dataset files:", path)

In [4]:
data = pd.read_csv(r"\.cache\kagglehub\datasets\caesarlupum\vertebralcolumndataset\versions\1\column_3C.csv")
data.shape

(310, 7)

In [5]:
data

,pelvic_incidence,pelvic_tilt,lumbar_lordosis_angle,sacral_slope,pelvic_radius,degree_spondylolisthesis,class
0,63.027817,22.552586,39.609117,40.475232,98.672917,-0.254400,Hernia
1,39.056951,10.060991,25.015378,28.995960,114.405425,4.564259,Hernia
2,68.832021,22.218482,50.092194,46.613539,105.985135,-3.530317,Hernia
3,69.297008,24.652878,44.311238,44.644130,101.868495,11.211523,Hernia
4,49.712859,9.652075,28.317406,40.060784,108.168725,7.918501,Hernia
...,...,...,...,...,...,...,...
305,47.903565,13.616688,36.000000,34.286877,117.449062,-4.245395,Normal
306,53.936748,20.721496,29.220534,33.215251,114.365845,-0.421010,Normal
307,61.446597,22.694968,46.170347,38.751628,125.670725,-2.707880,Normal
308,45.252792,8.693157,41.583126,36.559635,118.545842,0.214750,Normal


In [6]:
labels = data['class']
data = data.drop(columns=['class'])

In [7]:
Q1 = data.quantile(0.20) # Or X_scaled_df.quantile(0.25)
Q3 = data.quantile(0.80) # Or X_scaled_df.quantile(0.75)
IQR = Q3 - Q1
outlier_indices_iqr = ((data < (Q1 - 1.5 * IQR)) | (data > (Q3 + 1.5 * IQR))).any(axis=1)
# This gives a boolean Series indicating rows with at least one outlier feature

data = data[~outlier_indices_iqr]
y_train = labels[~outlier_indices_iqr]
print(f"Shape after removing outlier rows: {data.shape}")

Shape after removing outlier rows: (298, 6)


In [8]:
scaler = StandardScaler()
X_train = scaler.fit_transform(data)

In [9]:
results = []

In [2]:
# 4. Apply clustering algorithms
kmeans = KMeans(n_clusters=3, n_init=10, init='random', algorithm='lloyd', max_iter=10, random_state=42)  # Specify the number of clusters
kmeans_labels = kmeans.fit_predict(X_train)

ami_km = adjusted_mutual_info_score(y_train, kmeans_labels)
ari_km = adjusted_rand_score(y_train, kmeans_labels)

results.append(["KMeans", ami_km, ari_km])

NameError: name 'KMeans' is not defined

In [3]:
ncut = SpectralClustering(n_clusters=3, affinity='nearest_neighbors', assign_labels='kmeans', n_jobs=-1)  # Example parameters, tune as needed
ncut_labels = ncut.fit_predict(X_train)

ami_nc = adjusted_mutual_info_score(y_train, ncut_labels)
ari_nc = adjusted_rand_score(y_train, ncut_labels)

results.append(["NClust", ami_nc, ari_nc])

NameError: name 'SpectralClustering' is not defined

In [12]:
hclust = AgglomerativeClustering(n_clusters=3, linkage='single', metric='euclidean') # ward linkage is often a good choice
hclust_labels = hclust.fit_predict(X_train)

ami_hc = adjusted_mutual_info_score(y_train, hclust_labels)
ari_hc = adjusted_rand_score(y_train, hclust_labels)

results.append(["HClust(single)", ami_hc, ari_hc])

In [13]:
# Create Pandas DataFrame
results_df = pd.DataFrame(results, columns=["Model", "AMI", "ARI"])

print(results_df)

            Model       AMI       ARI
0          KMeans  0.358358  0.276795
1          NClust  0.317438  0.239874
2  HClust(single) -0.003432 -0.004031


In [4]:
import sys
import os

# 1. Get the path of the parent directory (Project_Root)
# '..' means "go up one level"
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))

# 2. Add it to Python's search path if not already there
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

# 3. Now you can import normally
from rlac import RLAC
from mdh import MDH

#print("Successfully imported models from:", parent_dir)

In [15]:
import warnings
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score

# Import your custom models
from rlac import RLAC
from mdh import MDH

# --- CONFIGURATION ---
# Target clusters for Aggregation dataset (should be 7)
n_clusters = len(set(y_train))

# RLAC Parameters
rlac_methods = [
    'depth_ratio', 'dip', 'holes', 'min_kurt', 'max_kurt', 
    'negentropy', 'skewness', 'fisher', 'hermite', 'friedman_tukey'
]
rlac_params = {
    'random_state': [32, 42, 43, 44, 45],
    'bw_adjust': [0.05, 0.1, 0.2, 0.3, 0.4, 0.5],
    'r': [None, 50, 100, 200, 500, 4883]
}
# MDH Parameters (Same as Anuran configuration)
mdh_config = {
    "h_multiplier": 1.0,
    "alphamax_val": 0.9,
    "alpha_steps": 5,
    "random_state": 42
}

results = []

print(f"\nStarting Benchmark on Aggregation Dataset (n={len(X_train)}, k={n_clusters})...")
print("-" * 80)

# ==========================================
# 1. RLAC LOOP
# ==========================================
for method in rlac_methods:
    for r_val in rlac_params['r']:
        for bw in rlac_params['bw_adjust']:
            for seed in rlac_params['random_state']:
                
                # 1. Print parameters BEFORE running
                param_str = f"r={r_val}, bw={bw}, s={seed}"
                print(f"\nRunning RLAC {method:<15} | {param_str} ... ", end="")
                
                try:
                    # 2. Instantiate & Fit
                    model = RLAC(
                        n_clusters=n_clusters,
                        method=method,
                        r=r_val,
                        bw_adjust=bw,
                        random_state=seed,
                        plot=False
                    )
                    
                    with warnings.catch_warnings():
                        warnings.filterwarnings("ignore", category=UserWarning)
                        model.fit(X_train)
                    
                    # 3. Evaluate
                    ami = adjusted_mutual_info_score(y_train, model.labels_)
                    ari = adjusted_rand_score(y_train, model.labels_)
                    
                    # 4. Log Success
                    print(f"Done (AMI: {ami:.4f})")
                    
                    results.append({
                        'Model': 'RLAC',
                        'Method': method,
                        'Params': param_str,
                        'AMI': ami,
                        'ARI': ari
                    })
                    
                except Exception as e:
                    print(f"FAILED. Error: {e}")
                    results.append({
                        'Model': 'RLAC', 'Method': method, 'Params': param_str,
                        'AMI': -1, 'ARI': -1
                    })

# ==========================================
# 2. MDH RUN
# ==========================================
print(f"\nRunning MDH {'Standard':<15} | h=1.0, a=0.9 ... ")
try:
    mdh_model = MDH(
        n_clusters=n_clusters,
        h_multiplier=mdh_config['h_multiplier'],
        alphamax_val=mdh_config['alphamax_val'],
        alpha_steps=mdh_config['alpha_steps'],
        random_state=mdh_config['random_state'],
        verbose=False,
        plot=False
    )
    
    mdh_model.fit(X_train)
    
    ami_mdh = adjusted_mutual_info_score(y_train, mdh_model.labels_)
    ari_mdh = adjusted_rand_score(y_train, mdh_model.labels_)
    
    print(f"Done (AMI: {ami_mdh:.4f})")
    
    results.append({
        'Model': 'MDH',
        'Method': 'Standard',
        'Params': 'Fixed',
        'AMI': ami_mdh,
        'ARI': ari_mdh
    })
    
except Exception as e:
    print(f"FAILED. Error: {e}")

# ==========================================
# 3. RESULTS TABLE
# ==========================================
print("\n" + "="*80)
print("FINAL RESULTS (ALL MODELS - SORTED BY AMI)")
print("="*80)

# 1. Create DataFrame
results_df = pd.DataFrame(results).sort_values(by='AMI', ascending=False)

# 2. Print FULL table (no truncation)
# index=False hides the row numbers for a cleaner look
print(results_df.to_string(index=False))


Starting Benchmark on Aggregation Dataset (n=298, k=3)...
--------------------------------------------------------------------------------

Running RLAC depth_ratio     | r=None, bw=0.05, s=32 ... [DEPTH_RATIO] Generating 1314 sparse random projections...
Starting Clustering: Target=3 clusters.
Iter 1: Split Cluster 0 (Size: 298) via Proj 105 | Score: 0.4546
Iter 2: Split Cluster 1 (Size: 276) via Proj 105 | Score: 0.3326
RLAC (depth_ratio) complete. Final clusters: 3
Done (AMI: 0.1121)

Running RLAC depth_ratio     | r=None, bw=0.05, s=42 ... [DEPTH_RATIO] Generating 1314 sparse random projections...
Starting Clustering: Target=3 clusters.
Iter 1: Split Cluster 0 (Size: 298) via Proj 780 | Score: 0.4546
Iter 2: Split Cluster 0 (Size: 276) via Proj 780 | Score: 0.3326
RLAC (depth_ratio) complete. Final clusters: 3
Done (AMI: 0.1121)

Running RLAC depth_ratio     | r=None, bw=0.05, s=43 ... [DEPTH_RATIO] Generating 1314 sparse random projections...
Starting Clustering: Target=3 cluster